Martial Wace take home challenges

In [21]:
# Challenge A, python script to retrieve SSL expiration date

# the quickest and easiest way is to simply query directly to www.mwam.com
import ssl
import socket
from datetime import datetime

hostname = 'www.mwam.com'
port = '443'
context = ssl.create_default_context()

with socket.create_connection((hostname, port)) as sock:
    with context.wrap_socket(sock, server_hostname = hostname) as ssock:
        certificate = ssock.getpeercert()


print(certificate['notAfter'])
certExpires = datetime.strptime(certificate['notAfter'], '%b %d %H:%M:%S %Y %Z')
print((certExpires - datetime.now()).days,  " days before expiration")


Jun 12 23:59:59 2022 GMT
77  days before expiration


In [24]:
# Alternatively retrieving from Qualys SSL Labs API, this is slower and takes about 2.4s to complete (with Cache on)
import requests
from datetime import datetime

exp_dates = []
response = requests.get(f"https://api.ssllabs.com/api/v3/analyze?host={hostname}&fromCache=on&all=done")
raw_certs = response.json()


for i in raw_certs['certs']:
    exp_dates.append(datetime.fromtimestamp(i['notAfter']/1000))
    

print(min(exp_dates).strftime('%b %d %H:%M:%S %Y %Z')) # find the earliest expiration date in the certificate chain
print((certExpires - datetime.now()).days,  " days before expiration")

Jun 13 00:59:59 2022 
77  days before expiration


Docker file is also on the repo. <br>
docker push cmskzhan/streamlits:WMChallenge_01

In [ ]:
# Challenge B
import smtplib
def send_notification(days_to_expire):
    smtp_port = 587
    smtp_server = "smtp.mwam.com"
    sender_email = "operations@mwam.com"
    receiver_email = "webmaster@mwam.com"
    password = "smptPassword"
    if days_to_expire== 1:
        days = "1 day"
    else:
        days = str(days_to_expire) + " days"
        
    message = """\
        Subject: Certificate Expiration
        The TLS Certificate for your site expires in {days}"""

    email_context = ssl.create_default_context()
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls(context = email_context)
        server.login(sender_email, password)
        server.sendmail(sender_email, 
                        receiver_email, 
                        message.format(days = days))

How would you scale this script and run it with resiliency to e.g. handle 1000s of domains? <br>
<br>
There are a few solutions, I've never tried any of them though.<br>
1. async http requests. Python has asyncio and aiohttp module for this <br>
2. multiple threads. Python has concurrent module with ThreadPoolExecutor <br>
3. multipile processes.  <br>
Alternatively, scale up thousands of pods on K8S is another solution. The docker image needs to be re-written obviously.

How would you monitor/alert on this service?<br>
<br>
This service would probably run on a cronjob. So checking the status of the service is quite straightforward. Furthermore, instead of sending out email, a function to create a webhook query to pagerduty or teams channel when expiration date is below certain threshold can be triggered, if need to be. <br>
For capturing exceptions, dumping logs to tools such as logstash, splunk, then setup alerts from there is a usual approach. 

What would you do to handle adding new domains to scan or certificate expiry events from your service? <br>
<br>
Instead of hardcoding the <hostname> in my code, it would come from the script's parameters or human input. Alternatively, put up an input file somewhere for the script to read.

After some time, your report requires more enhancements requested by the Tech team of the company. How would you handle these "continuous" requirement changes in a sustainable manner?<br>
<br>
Make the script more modula, such as create this service as a class for others to import, utilizing OOP concepts such as Polymorphism to make the code more flexible. 